In [1]:
pip install opencv-python dlib face_recognition numpy

  Using cached dlib-19.24.6.tar.gz (3.4 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [1 lines of output]
  ERROR: Can not execute `setup.py` since setuptools is not available in the build environment.
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

Encountered error while generating package metadata.

See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2

In [3]:
img = cv2.imread('PUPPY.jpg')

In [4]:
import os
print(os.getcwd())

C:\Users\Admin


In [5]:
img = cv2.imread('PUPPY.jpg')
if img is None:
    print("Image not found or path is incorrect.")
else:
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [6]:
img = cv2.imread('PUPPY.jpg')

In [7]:
import cv2

# Load the Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load the image using an absolute path
img = cv2.imread('PUPPY.jpg')

# Check if image was loaded successfully
if img is None:
    print("Error: Image not found or cannot be loaded.")
else:
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image with adjusted parameters
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=6)

    # Draw rectangles around the faces with reduced size
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x + int(w * 0.9), y + int(h * 0.9)), (255, 0, 0), 2)

    # Display the result
    cv2.imshow('Detected Faces', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [15]:
import tensorflow as tf
print(tf.__version__)

2.17.0


In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [26]:
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, Flatten, Lambda, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Lambda

In [27]:
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Lambda, Conv2D, MaxPooling2D
from tensorflow.keras import backend as K

# Load your dataset
def load_data(dataset_path):
    images = []
    labels = []
    for label in os.listdir(dataset_path):
        for img_file in os.listdir(os.path.join(dataset_path, label)):
            img_path = os.path.join(dataset_path, label, img_file)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (105, 105))  # Resize to fit the model
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

# Create pairs of images and labels
def create_pairs(images, labels):
    pairs = []
    pair_labels = []
    unique_labels = np.unique(labels)
    for label in unique_labels:
        # Get images of the same person
        same_person = images[labels == label]
        for i in range(len(same_person)):
            for j in range(i + 1, len(same_person)):
                pairs.append((same_person[i], same_person[j]))
                pair_labels.append(1)  # Same person label
            # Get images of different people
            different_people = images[labels != label]
            for img in different_people:
                pairs.append((same_person[i], img))
                pair_labels.append(0)  # Different person label
    return np.array(pairs), np.array(pair_labels)

# Build the Siamese network model
def build_siamese_model():
    input_shape = (105, 105, 3)

    # Define the base network
    base_network = Sequential()
    base_network.add(Conv2D(64, (10, 10), activation='relu', input_shape=input_shape))
    base_network.add(MaxPooling2D())
    base_network.add(Conv2D(128, (7, 7), activation='relu'))
    base_network.add(MaxPooling2D())
    base_network.add(Conv2D(128, (4, 4), activation='relu'))
    base_network.add(MaxPooling2D())
    base_network.add(Flatten())
    base_network.add(Dense(256, activation='sigmoid'))

    # Create two input tensors
    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)

    # Create the outputs of the base network
    output_a = base_network(input_a)
    output_b = base_network(input_b)

    # Calculate the distance between the outputs
    distance = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))([output_a, output_b])
    output = Dense(1, activation='sigmoid')(distance)

    model = Model(inputs=[input_a, input_b], outputs=output)
    return model

# Main function to train the model
def train_model(dataset_path):
    images, labels = load_data(dataset_path)
    pairs, pair_labels = create_pairs(images, labels)

    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(pairs, pair_labels, test_size=0.2, random_state=42)

    # Build the model
    model = build_siamese_model()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model
    model.fit([X_train[:, 0], X_train[:, 1]], y_train, batch_size=32, epochs=10)

    # Evaluate the model
    loss, accuracy = model.evaluate([X_test[:, 0], X_test[:, 1]], y_test)
    print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

# Set the dataset path and train the model
dataset_path = (r'C:\Users\Admin\OneDrive\Desktop\my_face_dataset')
train_model(dataset_path)


C:\Users\Admin\anaconda3\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.5000 - loss: 0.6021
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 0.2500 - loss: 0.7660
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 0.7500 - loss: 0.6392
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - accuracy: 1.0000 - loss: 0.5587
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - accuracy: 1.0000 - loss: 0.5501
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 1.0000 - loss: 0.6048
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - accuracy: 0.7500 - loss: 0.6313
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.5000 - loss: 0.6860
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - accuracy: 0.7500 - loss: 0.6377
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.7500 - loss: 0.6356
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step - accuracy: 1.0000 - loss: 0.6455
Test Loss: 0.6454880237579346, Test Accuracy: 1.0
